In [3]:
# 이 파일은 처음 project_data1, 2, 3 파일을 만들 때 사용한 코드입니다.

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pandas as pd

### 1. '하이퍼클로바X' 관련 기사 크롤링하기 (11월 1일 ~)

In [4]:
# 검색어 설정 : 하이퍼클로바X, 클로바X, 네이버 큐(CUE)
keyword = '네이버 하이퍼클로바X'
news_data = []

# 크롤링 기간 설정 (2023.11.01 ~ 2023.11.10)
crowl_url = f"https://search.naver.com/search.naver?where=news&query={keyword}&sm=tab_opt&sort=1&photo=0&field=0&pd=3&ds=2023.11.01&de=2023.11.10&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom20231105to20231105&is_sug_officeid=0&office_category=0&service_area=0"

# 브라우저 열기 & 웹페이지 접속
browser = webdriver.Chrome()
browser.get(crowl_url)
time.sleep(2)

soup = BeautifulSoup(browser.page_source, "html.parser")

# 크롤링해야 하는 기사 링크 담아놓을 리스트
news_link = []

# ainfo = 결과 기사의 언론사, '네이버뉴스' 태그 리스트
ainfo = browser.find_elements("css selector", "ul.list_news li.bx div.info_group a.info")

# 네이버뉴스 달려있는 기사링크만 추출!
for i,a in enumerate(ainfo):
    if a.text == '네이버뉴스':
        link = soup.select("div.info_group a.info")[i]['href']
        news_link.append(link)

# 다음 페이지 버튼이 활성화되어 있는 경우
# = 다음 페이지가 존재하는 경우
# = is_next_page 값이 'false'인 경우
is_next_page = soup.select("a.btn_next")[0]['aria-disabled']
btn_next = browser.find_elements("css selector", "a.btn_next")[0]

while True:

    if is_next_page == 'false':
        btn_next.click()
        time.sleep(1)

        ainfo = browser.find_elements("css selector", "ul.list_news li.bx div.info_group a.info")
        soup = BeautifulSoup(browser.page_source, "html.parser")
        for i,a in enumerate(ainfo):
            if a.text == '네이버뉴스':
                link = soup.select("div.info_group a.info")[i]['href']
                news_link.append(link)

        is_next_page = soup.select("a.btn_next")[0]['aria-disabled']
        btn_next = browser.find_elements("css selector", "a.btn_next")[0]

    elif is_next_page == 'true':
        break
    else:
        break


# 링크 하나씩 넣어서 크롤링 시작
for news in news_link:
    browser.get(news)
    time.sleep(1)

    # 기사 제목
    title = browser.find_elements("css selector", "h2#title_area")[0].text

    # 언론사
    soup = BeautifulSoup(browser.page_source, "html.parser")
    company = soup.select("a.media_end_head_top_logo._LAZY_LOADING_ERROR_HIDE img")[0]['title']

    # 최종입력시간 / 입력시간
    write_time = browser.find_elements("css selector", "div.media_end_head_info_datestamp_bunch")
    # len(write_time) == 2 이면 입력시간, 최종입력시간 둘다, len(write_time) == 1 이면 입력시간 = 최종입력시간
    if len(write_time) == 1:
        first = soup.select("span.media_end_head_info_datestamp_time._ARTICLE_DATE_TIME")[0]['data-date-time']
        enter = soup.select("span.media_end_head_info_datestamp_time._ARTICLE_DATE_TIME")[0]['data-date-time']
    elif len(write_time) == 2:
        first = soup.select("span.media_end_head_info_datestamp_time._ARTICLE_DATE_TIME")[0]['data-date-time']
        enter = soup.select("span.media_end_head_info_datestamp_time._ARTICLE_MODIFY_DATE_TIME")[0]['data-modify-date-time']
    
    news_content = browser.find_elements("css selector", "div#newsct_article")[0].text.replace("\n", ' ').replace("\'", "")

    data = [title, company, enter, first, news, news_content]
    news_data.append(data)

# 겹치는 기사는 제거
tuple_origin = [tuple(i) for i in news_data]
unique_data = [list(j) for j in set(tuple_origin)]

In [5]:
len(unique_data)

152

In [6]:
df = pd.DataFrame(unique_data)
df.columns = ['제목', '언론사', '최종입력시간', '입력시간', 'URL', '본문']
df = df.sort_values(by=['최종입력시간', '입력시간'], ascending=[False, False]).reset_index(drop=True)
df

,제목,언론사,최종입력시간,입력시간,URL,본문
0,'K콘텐츠' 넘어 'K플랫폼' 쏟아지길 [기자수첩],파이낸셜뉴스,2023-11-16 11:00:06,2023-11-09 16:17:02,https://n.news.naver.com/mnews/article/014/000...,"[파이낸셜뉴스] ""오징어게임이 넷플릭스에서 1위를 하고 BTS(방탄소년단)가 빌보드..."
1,"[인터뷰] '링크' 최찬열 대표 ""아이언맨의 '자비스' 만들려면…""",주간조선,2023-11-14 10:31:01,2023-11-07 04:01:01,https://n.news.naver.com/mnews/article/053/000...,photo 이신영 영상미디어 기자 특정 사유에 따른 이혼소송 승소 가능성 적합한...
2,[IS현장] 자연이 식히고 로봇이 나르고…네이버가 세종에 세운 최첨단 데이터 방주,일간스포츠,2023-11-10 20:20:01,2023-11-10 07:00:04,https://n.news.naver.com/mnews/article/241/000...,아시아 최대 규모 자랑하는 각 세종 축구장 41개·국립중앙도서관 100만배 외기로 ...
3,"AI 기술패권 경쟁에 뛰어든 韓 …순항 시작한 네이버, 잠시 숨고른 카카오",디지털데일리,2023-11-10 18:17:01,2023-11-10 18:17:01,https://n.news.naver.com/mnews/article/138/000...,"[ⓒ 네이버·카카오] [디지털데일리 이나연 기자] 출발선은 같았지만, 속도는 달..."
4,[DD퇴근길] KT-LGU+ 누가 2위일까…넥슨 독주에 크래프톤 약진,디지털데일리,2023-11-10 17:31:05,2023-11-10 17:31:05,https://n.news.naver.com/mnews/article/138/000...,디지털데일리가 퇴근 즈음해서 읽을 수 있는 [DD퇴근길] 코너를 마련했습니다. 하루...
...,...,...,...,...,...,...
147,"삼정KPMG-오브젠, 초거대 AI 기반 비즈니스 혁신 '맞손'",파이낸셜뉴스,2023-11-02 08:32:03,2023-11-02 08:32:03,https://n.news.naver.com/mnews/article/014/000...,양사 협력으로 초거대 AI 활용한 새 비즈니스 기회 창출 기대 생성형 AI 활용한 ...
148,3분기 실적에 네이버 '웃고' 카카오 '울고',뉴시스,2023-11-01 18:08:15,2023-11-01 16:02:08,https://n.news.naver.com/mnews/article/003/001...,"경기 침체 지속되며 네카오 광고 성장 둔화 네이버, 커머스 견조·비용 통제로 선방…..."
149,"이영해 울산시의원, AI 활용해 5분 자유발언 진행",연합뉴스,2023-11-01 11:31:16,2023-11-01 11:30:47,https://n.news.naver.com/mnews/article/001/001...,수돗물 이용 활성화를 위한 울산시 인식 전환 주문 AI 이용 5분 자유발언을 한...
150,네카오 3분기 실적도 엇갈릴 듯…양대 플랫폼 체제 '흔들',연합뉴스,2023-11-01 09:04:12,2023-11-01 09:03:14,https://n.news.naver.com/mnews/article/001/001...,작년비 분기별 영업익 네이버 증가·카카오 감소 지속 하이퍼클로바X로 성장 견인 VS...


In [7]:
# 이후에 new 컬럼이 필요해져서 아래 코드로 'new' 컬럼을 생성하여 저장하였습니다. 
df.insert(0, 'new', ' ')
df.to_excel('./project_data1.xlsx', index= False)

### 2. '클로바X' 관련 기사 크롤링하기 (11월 1일 ~)

In [8]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pandas as pd

# 검색어 설정 : 하이퍼클로바X, 클로바X, 네이버 큐(CUE)
keyword = '네이버 클로바X'
news_data = []

# 크롤링 기간 설정 (2023.11.01 ~ 2023.11.10)
crowl_url = f"https://search.naver.com/search.naver?where=news&query={keyword}&sm=tab_opt&sort=1&photo=0&field=0&pd=3&ds=2023.11.01&de=2023.11.10&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom20231105to20231105&is_sug_officeid=0&office_category=0&service_area=0"

# 브라우저 열기 & 웹페이지 접속
browser = webdriver.Chrome()
browser.get(crowl_url)
time.sleep(2)

soup = BeautifulSoup(browser.page_source, "html.parser")

# 크롤링해야 하는 기사 링크 담아놓을 리스트
news_link = []

# ainfo = 결과 기사의 언론사, '네이버뉴스' 태그 리스트
ainfo = browser.find_elements("css selector", "ul.list_news li.bx div.info_group a.info")

# 네이버뉴스 달려있는 기사링크만 추출!
for i,a in enumerate(ainfo):
    if a.text == '네이버뉴스':
        link = soup.select("div.info_group a.info")[i]['href']
        news_link.append(link)

# 다음 페이지 버튼이 활성화되어 있는 경우
# = 다음 페이지가 존재하는 경우
# = is_next_page 값이 'false'인 경우
is_next_page = soup.select("a.btn_next")[0]['aria-disabled']
btn_next = browser.find_elements("css selector", "a.btn_next")[0]

while True:

    if is_next_page == 'false':
        btn_next.click()
        time.sleep(1)

        ainfo = browser.find_elements("css selector", "ul.list_news li.bx div.info_group a.info")
        soup = BeautifulSoup(browser.page_source, "html.parser")
        for i,a in enumerate(ainfo):
            if a.text == '네이버뉴스':
                link = soup.select("div.info_group a.info")[i]['href']
                news_link.append(link)

        is_next_page = soup.select("a.btn_next")[0]['aria-disabled']
        btn_next = browser.find_elements("css selector", "a.btn_next")[0]

    elif is_next_page == 'true':
        break
    else:
        break


# 링크 하나씩 넣어서 크롤링 시작
for news in news_link:
    browser.get(news)
    time.sleep(1)

    # 기사 제목
    title = browser.find_elements("css selector", "h2#title_area")[0].text

    # 언론사
    soup = BeautifulSoup(browser.page_source, "html.parser")
    company = soup.select("a.media_end_head_top_logo._LAZY_LOADING_ERROR_HIDE img")[0]['title']

    # 최종 입력시간
    write_time = browser.find_elements("css selector", "div.media_end_head_info_datestamp_bunch")
    # len(write_time) == 2 이면 수정한 시간 write_time[1]을 가져오고, len(write_time) == 1 이면 입력시간
    if len(write_time) == 1:
        first = soup.select("span.media_end_head_info_datestamp_time._ARTICLE_DATE_TIME")[0]['data-date-time']
        enter = soup.select("span.media_end_head_info_datestamp_time._ARTICLE_DATE_TIME")[0]['data-date-time']
    elif len(write_time) == 2:
        first = soup.select("span.media_end_head_info_datestamp_time._ARTICLE_DATE_TIME")[0]['data-date-time']
        enter = soup.select("span.media_end_head_info_datestamp_time._ARTICLE_MODIFY_DATE_TIME")[0]['data-modify-date-time']
    
    news_content = browser.find_elements("css selector", "div#newsct_article")[0].text.replace("\n", ' ').replace("\'", "")

    data = [title, company, enter, first, news, news_content]
    news_data.append(data)

# 겹치는 기사는 제거
tuple_origin = [tuple(i) for i in news_data]
unique_data = [list(j) for j in set(tuple_origin)]

In [9]:
len(unique_data)

154

In [10]:
df = pd.DataFrame(unique_data)
df.columns = ['제목', '언론사', '최종입력시간', '입력시간', 'URL', '본문']
df = df.sort_values(by=['최종입력시간', '입력시간'], ascending=[False, False]).reset_index(drop=True)
df

,제목,언론사,최종입력시간,입력시간,URL,본문
0,'K콘텐츠' 넘어 'K플랫폼' 쏟아지길 [기자수첩],파이낸셜뉴스,2023-11-16 11:00:06,2023-11-09 16:17:02,https://n.news.naver.com/mnews/article/014/000...,"[파이낸셜뉴스] ""오징어게임이 넷플릭스에서 1위를 하고 BTS(방탄소년단)가 빌보드..."
1,"[인터뷰] '링크' 최찬열 대표 ""아이언맨의 '자비스' 만들려면…""",주간조선,2023-11-14 10:31:01,2023-11-07 04:01:01,https://n.news.naver.com/mnews/article/053/000...,photo 이신영 영상미디어 기자 특정 사유에 따른 이혼소송 승소 가능성 적합한...
2,[IS현장] 자연이 식히고 로봇이 나르고…네이버가 세종에 세운 최첨단 데이터 방주,일간스포츠,2023-11-10 20:20:01,2023-11-10 07:00:04,https://n.news.naver.com/mnews/article/241/000...,아시아 최대 규모 자랑하는 각 세종 축구장 41개·국립중앙도서관 100만배 외기로 ...
3,"AI 기술패권 경쟁에 뛰어든 韓 …순항 시작한 네이버, 잠시 숨고른 카카오",디지털데일리,2023-11-10 18:17:01,2023-11-10 18:17:01,https://n.news.naver.com/mnews/article/138/000...,"[ⓒ 네이버·카카오] [디지털데일리 이나연 기자] 출발선은 같았지만, 속도는 달..."
4,[DD퇴근길] KT-LGU+ 누가 2위일까…넥슨 독주에 크래프톤 약진,디지털데일리,2023-11-10 17:31:05,2023-11-10 17:31:05,https://n.news.naver.com/mnews/article/138/000...,디지털데일리가 퇴근 즈음해서 읽을 수 있는 [DD퇴근길] 코너를 마련했습니다. 하루...
...,...,...,...,...,...,...
149,"삼정KPMG-오브젠, 초거대 AI 기반 비즈니스 혁신 '맞손'",파이낸셜뉴스,2023-11-02 08:32:03,2023-11-02 08:32:03,https://n.news.naver.com/mnews/article/014/000...,양사 협력으로 초거대 AI 활용한 새 비즈니스 기회 창출 기대 생성형 AI 활용한 ...
150,3분기 실적에 네이버 '웃고' 카카오 '울고',뉴시스,2023-11-01 18:08:15,2023-11-01 16:02:08,https://n.news.naver.com/mnews/article/003/001...,"경기 침체 지속되며 네카오 광고 성장 둔화 네이버, 커머스 견조·비용 통제로 선방…..."
151,"이영해 울산시의원, AI 활용해 5분 자유발언 진행",연합뉴스,2023-11-01 11:31:16,2023-11-01 11:30:47,https://n.news.naver.com/mnews/article/001/001...,수돗물 이용 활성화를 위한 울산시 인식 전환 주문 AI 이용 5분 자유발언을 한...
152,네카오 3분기 실적도 엇갈릴 듯…양대 플랫폼 체제 '흔들',연합뉴스,2023-11-01 09:04:12,2023-11-01 09:03:14,https://n.news.naver.com/mnews/article/001/001...,작년비 분기별 영업익 네이버 증가·카카오 감소 지속 하이퍼클로바X로 성장 견인 VS...


In [11]:
df.insert(0, 'new', ' ')
df.to_excel('./project_data2.xlsx', index= False)

### 3. '큐' 관련 기사 크롤링하기 (11월 1일 ~)

In [12]:
# 검색어 설정 : 하이퍼클로바X, 클로바X, 네이버 큐(CUE)
keyword = '네이버 큐'
news_data = []

# 크롤링 기간 설정 (2023.11.01 ~ 2023.11.10)
crowl_url = f"https://search.naver.com/search.naver?where=news&query={keyword}&sm=tab_opt&sort=1&photo=0&field=0&pd=3&ds=2023.11.01&de=2023.11.10&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom20231105to20231105&is_sug_officeid=0&office_category=0&service_area=0"

# 브라우저 열기 & 웹페이지 접속
browser = webdriver.Chrome()
browser.get(crowl_url)
time.sleep(2)

soup = BeautifulSoup(browser.page_source, "html.parser")

# 크롤링해야 하는 기사 링크 담아놓을 리스트
news_link = []

# ainfo = 결과 기사의 언론사, '네이버뉴스' 태그 리스트
ainfo = browser.find_elements("css selector", "ul.list_news li.bx div.info_group a.info")

# 네이버뉴스 달려있는 기사링크만 추출!
for i,a in enumerate(ainfo):
    if a.text == '네이버뉴스':
        link = soup.select("div.info_group a.info")[i]['href']
        news_link.append(link)

# 다음 페이지 버튼이 활성화되어 있는 경우
# = 다음 페이지가 존재하는 경우
# = is_next_page 값이 'false'인 경우
is_next_page = soup.select("a.btn_next")[0]['aria-disabled']
btn_next = browser.find_elements("css selector", "a.btn_next")[0]

while True:

    if is_next_page == 'false':
        btn_next.click()
        time.sleep(1)

        ainfo = browser.find_elements("css selector", "ul.list_news li.bx div.info_group a.info")
        soup = BeautifulSoup(browser.page_source, "html.parser")
        for i,a in enumerate(ainfo):
            if a.text == '네이버뉴스':
                link = soup.select("div.info_group a.info")[i]['href']
                news_link.append(link)

        is_next_page = soup.select("a.btn_next")[0]['aria-disabled']
        btn_next = browser.find_elements("css selector", "a.btn_next")[0]

    elif is_next_page == 'true':
        break
    else:
        break


# 링크 하나씩 넣어서 크롤링 시작
for news in news_link:
    browser.get(news)
    time.sleep(1)

    # 기사 제목
    title = browser.find_elements("css selector", "h2#title_area")[0].text

    # 언론사
    soup = BeautifulSoup(browser.page_source, "html.parser")
    company = soup.select("a.media_end_head_top_logo._LAZY_LOADING_ERROR_HIDE img")[0]['title']

    # 최종 입력시간
    write_time = browser.find_elements("css selector", "div.media_end_head_info_datestamp_bunch")
    # len(write_time) == 2 이면 수정한 시간 write_time[1]을 가져오고, len(write_time) == 1 이면 입력시간
    if len(write_time) == 1:
        first = soup.select("span.media_end_head_info_datestamp_time._ARTICLE_DATE_TIME")[0]['data-date-time']
        enter = soup.select("span.media_end_head_info_datestamp_time._ARTICLE_DATE_TIME")[0]['data-date-time']
    elif len(write_time) == 2:
        first = soup.select("span.media_end_head_info_datestamp_time._ARTICLE_DATE_TIME")[0]['data-date-time']
        enter = soup.select("span.media_end_head_info_datestamp_time._ARTICLE_MODIFY_DATE_TIME")[0]['data-modify-date-time']
    
    news_content = browser.find_elements("css selector", "div#newsct_article")[0].text.replace("\n", ' ').replace("\'", "")

    data = [title, company, enter, first, news, news_content]
    news_data.append(data)

# 겹치는 기사는 제거
tuple_origin = [tuple(i) for i in news_data]
unique_data = [list(j) for j in set(tuple_origin)]

In [13]:
len(unique_data)

63

In [14]:
df = pd.DataFrame(unique_data)
df.columns = ['제목', '언론사', '최종입력시간', '입력시간', 'URL', '본문']
df = df.sort_values(by=['최종입력시간', '입력시간'], ascending=[False, False]).reset_index(drop=True)
df

,제목,언론사,최종입력시간,입력시간,URL,본문
0,"AI 기술패권 경쟁에 뛰어든 韓 …순항 시작한 네이버, 잠시 숨고른 카카오",디지털데일리,2023-11-10 18:17:01,2023-11-10 18:17:01,https://n.news.naver.com/mnews/article/138/000...,"[ⓒ 네이버·카카오] [디지털데일리 이나연 기자] 출발선은 같았지만, 속도는 달..."
1,"‘웅장한’ 네이버 데이터센터 각 세종, 협소하다고 느낀 까닭 [가봤어요]",이코노미스트,2023-11-10 15:39:01,2023-11-08 17:28:01,https://n.news.naver.com/mnews/article/243/000...,"‘아시아 최대’ 데이터센터 각 세종, 48개월 준비 끝에 운영 본격화 ‘각 춘천’ ..."
2,"[사고]생성형 AI가 변화시킬 미래, 먼저 만나보세요[ECF for 2024]",이데일리,2023-11-10 13:17:01,2023-11-10 13:17:01,https://n.news.naver.com/mnews/article/018/000...,제10회 이데일리 IT컨버전스포럼 개최 21일 오전 10시 여의도 FKI타워 컨퍼런...
3,한국 출시 구글 생성형 인공지능 기반 검색 서비스가 미칠 영향은?,미디어오늘,2023-11-10 11:48:00,2023-11-10 11:40:01,https://n.news.naver.com/mnews/article/006/000...,생성형 AI 검색 경쟁 나선 검색엔진들 환각현상 등 한계 있어 테스트 버전 보조적 ...
4,나란히 역대 최고 매출… 수익성은 네이버 ‘웃고’ 카카오 ‘울고’,세계일보,2023-11-09 20:18:06,2023-11-09 20:18:06,https://n.news.naver.com/mnews/article/022/000...,양대 플랫폼 3분기 엇갈린 실적 네이버 매출 18.9%↑ 2조4453억 영업익도 ...
...,...,...,...,...,...,...
58,"[컨콜] 네이버 ""생성형 AI 검색 '큐:' 내년 모바일 적용""",뉴시스,2023-11-03 09:22:05,2023-11-03 09:22:05,https://n.news.naver.com/mnews/article/003/001...,11월부터 PC 통합 검색에 부분 적용 예정 내년 모바일 환경에 도입…멀티모달 기술...
59,‘카카오’만 붙이면 성공하던 시대 끝나,매경이코노미,2023-11-02 21:02:03,2023-11-02 21:02:03,https://n.news.naver.com/mnews/article/024/000...,카카오 주요 사업 어디로 ‘카톡’ 너마저…신사업·해외 공략 차질 2010년대 중...
60,"""접속화면이 다 다르다고?""…AI 검색 적용 '네이버 서비스 진화'",뉴스1,2023-11-02 16:38:01,2023-11-02 16:37:06,https://n.news.naver.com/mnews/article/421/000...,"""AI 콘텐츠 추천 시스템에 하이퍼클로바X 더 많이 적용…추천 고도화"" 네이버 웹사..."
61,"광고시장 위축됐는데…네이버 이익 확 늘어날것 같은 예감이, 왜?",매일경제,2023-11-02 16:32:00,2023-11-02 16:32:00,https://n.news.naver.com/mnews/article/009/000...,"네이버, 실적 발표 D-1 광고 매출 성장 둔화 전망 광고수익 빈자리 커머스로 신규..."


In [15]:
df.insert(0, 'new', ' ')
df.to_excel('./project_data3.xlsx', index= False)